<h3>Import required libraries</h3>

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

<h3>Load the Pre-trained VGG19 Model</h3>

In [6]:
# Number of classes for the food recognition model
num_of_foods = 12

# Load the VGG19 model pre-trained on ImageNet, without the top layer
base_model = VGG19(
    weights='imagenet',       # Use weights pre-trained on ImageNet
    include_top=False,        # Do not include the top (fully connected) layers
    input_shape=(224, 224, 3) # Input shape must match VGG19's requirements
)

# Freeze the base model to prevent its weights from being updated during training
base_model.trainable = False

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


<h3>Add Custom Classification Layers</h3>
<h4>Add custom layers to the base model for classification</h4>

In [7]:
# Functional API approach to build the model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)       # Pass inputs through the VGG19 model

# Add global average pooling to reduce dimensions
x = layers.GlobalAveragePooling2D()(x)

# Add a dense layer for further learning
x = layers.Dense(128, activation='relu')(x)

# Output layer with softmax activation for multi-class classification
outputs = layers.Dense(num_of_foods, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs, outputs)

<h3>Compile the Model</h3>
<h4>Compile the Model using an appropriate optimizer, loss function, and metrics</h4>

In [16]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if labels are integers
    metrics=['accuracy']
)

# Display the model summary
model.summary()
mary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 7, 7, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,091,596 (76.64 MB)

 Trainable params: 67,212 (262.55 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

NameError: name 'mary' is not defined

<h3>Prepare the Data using ImageDataGenerator</h3>

In [17]:
# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,        # Rescale pixel values from [0, 255] to [0, 1]
    rotation_range=20,     # Data augmentation options
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2   # Use 20% of the data for validation
)

# Path to your training dataset directory
data_dir = 'D:/food_dataset'  # Replace with the actual path

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Resize images to match VGG19 input size
    batch_size=32,
    class_mode='sparse',     # Use 'sparse' if labels are integers
    subset='training'
)

# Create validation data generator
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 5038 images belonging to 12 classes.
Found 1254 images belonging to 12 classes.


<h3>Train the Model</h3>

In [18]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # You can change the number of epochs based on your requirements
    validation_data=validation_generator
)

Epoch 1/10


D:\apps\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


158/158 ━━━━━━━━━━━━━━━━━━━━ 3773s 23s/step - accuracy: 0.3652 - loss: 2.0999 - val_accuracy: 0.5750 - val_loss: 1.4176
Epoch 2/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 3095s 19s/step - accuracy: 0.6160 - loss: 1.3153 - val_accuracy: 0.6324 - val_loss: 1.2120
Epoch 3/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 3128s 19s/step - accuracy: 0.6697 - loss: 1.0992 - val_accuracy: 0.6683 - val_loss: 1.0306
Epoch 4/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 3195s 20s/step - accuracy: 0.7011 - loss: 0.9682 - val_accuracy: 0.7026 - val_loss: 0.9146
Epoch 5/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 3293s 20s/step - accuracy: 0.7107 - loss: 0.9097 - val_accuracy: 0.7081 - val_loss: 0.8801
Epoch 6/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 3633s 22s/step - accuracy: 0.7300 - loss: 0.8775 - val_accuracy: 0.7185 - val_loss: 0.8555
Epoch 7/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 3074s 19s/step - accuracy: 0.7571 - loss: 0.8102 - val_accuracy: 0.7329 - val_loss: 0.7991
Epoch 8/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 3119s 19s/step - accuracy: 0.7714 - loss: 0.7477 - val

<h3>Evaluate the Model</h3>

In [19]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


40/40 ━━━━━━━━━━━━━━━━━━━━ 661s 16s/step - accuracy: 0.7653 - loss: 0.7371
Validation Accuracy: 76.95%
